In [41]:
import pandas as pd
import numpy as np

In [42]:
df = pd.read_csv('ipl.csv')
df.head()

,PLAYER,Mat.x,Inns.x,NO,Runs.x,HS,Avg.x,BF,SR.x,X100,...,Ov,Runs.y,Wkts,BBI,Avg.y,Econ,SR.y,X4w,X5w,y
0,Aaron Finch,10,9,1,134,46,16.75,100,134.00,0,...,0.0,0,0,0,0,0.0,0,0,0,0
1,AB de Villiers,12,11,2,480,90,53.33,275,174.54,0,...,0.0,0,0,0,0,0.0,0,0,0,0
2,Abhishek Sharma,3,3,2,63,46,63,33,190.90,0,...,0.0,0,0,0,0,0.0,0,0,0,0
3,Ajinkya Rahane,15,14,1,370,65,28.46,313,118.21,0,...,0.0,0,0,0,0,0.0,0,0,0,0
4,Alex Hales,6,6,0,148,45,24.66,118,125.42,0,...,0.0,0,0,0,0,0.0,0,0,0,0


In [43]:
df.shape

(143, 25)

In [44]:
df.columns

Index(['PLAYER', 'Mat.x', 'Inns.x', 'NO', 'Runs.x', 'HS', 'Avg.x', 'BF',
       'SR.x', 'X100', 'X50', 'X4s', 'X6s', 'Mat.y', 'Inns.y', 'Ov', 'Runs.y',
       'Wkts', 'BBI', 'Avg.y', 'Econ', 'SR.y', 'X4w', 'X5w', 'y'],
      dtype='object')

In [60]:
df = pd.read_csv('ipl.csv')
df.head()
df_names = df['PLAYER']
df_replace = df.drop('PLAYER', axis=1)
df_replace = df_replace.drop( ['y', 'BBI'], axis=1)
df_replace.replace('-', 0, inplace=True)
# get column type for each column
df_replace = df_replace.apply(pd.to_numeric, errors='coerce')
df_replace = (df_replace - df_replace.mean()) / df_replace.std()
df_replace.to_csv('ipl_norm.csv', index=False)
df_replace.replace(np.nan, 0, inplace=True)
X = df_replace.to_numpy()

In [67]:
class MyKMeans(object):

    def __init__(self, k, iterations = 20):
        self.iterations = iterations
        self.k = k

    def cosine_similarity_distance(self, a, b):
        return 1 - np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    
    def fit(self, X):
        self.centroids = {}
        # generate 5 random centroids
        idx = np.random.choice(len(X), self.k, replace=False)
        self.centroids = X[idx]
        print(self.centroids)
        last_centroids = np.zeros(self.centroids.shape)
        iters = 0
        while np.not_equal(self.centroids, last_centroids).any() and iters < self.iterations:
            clustered_points = [[] for i in range(self.k)]
            for point in X:
                distances = [self.cosine_similarity_distance(point, centroid) for centroid in self.centroids]
                #print(distances)
                clustered_points[np.argmin(distances)].append(point)
            last_centroids = self.centroids
            self.centroids = [np.mean(cluster, axis=0) for cluster in clustered_points]
            for i in range(self.k):
                if len(clustered_points[i]) == 0:
                    self.centroids[i] = last_centroids[i]
            iters += 1

    def evaluate(self, X):
        clustered_points = [[] for i in range(self.k)]
        cluster_indices = []
        for point in X:
            distances = [self.cosine_similarity_distance(point, centroid) for centroid in self.centroids]
            clustered_points[np.argmin(distances)].append(point)
            cluster_indices.append(np.argmin(distances))
        return clustered_points, cluster_indices


In [75]:
kmeans = MyKMeans(3)
kmeans.fit(X)
clusters, cluster_indices = kmeans.evaluate(X)
cluster_names = [[] for i in range(3)]
print(cluster_indices)
for i in range(len(cluster_indices)):
    cluster_names[cluster_indices[i]].append(df_names.iloc[i])
print(cluster_names)
# print(df_names)
# for i in range(3):
#     print('Cluster {} has {} points'.format(i, len(clusters[i])))
#     for i in range(len(clusters[i])):
#         print(df_names.iloc[cluster_indices[i]], end=' ')

[[ 1.43365052  0.72485875  2.91436195  0.04929473  1.08997494  1.07237992
  -0.03348637  0.91989773 -0.15948552  0.20285888 -0.20720149  0.44690741
   1.84185046  2.04060872  1.97938585  2.57557071  1.58562419  0.86480693
   0.81238665  0.72827842 -0.18108783 -0.0836242 ]
 [-1.19892765 -1.09364653 -0.76829277 -0.75420537 -1.03704017 -1.00390166
  -0.79001086 -1.38566033 -0.15948552 -0.48782731 -0.68606715 -0.69024134
  -0.63956584 -0.55818177 -0.66670647 -0.47006043 -0.44492141  0.60142324
   2.3060908  -0.12140749 -0.18108783 -0.0836242 ]
 [ 1.43365052  1.81596192 -0.76829277  1.24599701  0.65206006  0.28781054
   1.41305496  0.58523915 -0.15948552  0.20285888  1.10967907  1.69777104
  -0.99405388 -0.92943756 -0.85267681 -0.89306475 -0.78334568 -0.93537567
  -1.31598149 -0.97704566 -0.18108783 -0.0836242 ]]
[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 1, 0, 2, 1, 2, 1, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 1, 2, 2, 1, 2, 0, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0, 0, 1, 1,